<h1>NN Model</h1>

In [1]:
from pycocotools.coco import COCO
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as functions
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import re

/shared/storage/cs/studentscratch/js3921/python_environments/envs/project_env/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/shared/storage/cs/studentscratch/js3921/python_environments/envs/project_env/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
DATADIR = "cocodoom/"
USED_RUNS = ["run1", "run2", "run3"]

dataSplit, TRAIN_RUN = "run-full-train", "run1"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [3]:
coco_train = COCO(annFile)

loading annotations into memory...
Done (t=25.89s)
creating index...
index created!


In [4]:
dataSplit, VAL_RUN = "run-full-val", "run2"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [5]:
coco_val = COCO(annFile)

loading annotations into memory...
Done (t=23.56s)
creating index...
index created!


In [6]:
dataSplit, TEST_RUN = "run-full-test", "run3"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [7]:
coco_test = COCO(annFile)

loading annotations into memory...
Done (t=13.11s)
creating index...
index created!


In [8]:
player_positions = {"run1":{}, "run2":{}, "run3":{}}

for run in USED_RUNS:
    with open(DATADIR+run+"/log.txt", 'r') as log_file:
        for line in log_file:
            if "player" in line:
                line = line.strip()
                tic, stats = line.split("player:")
                x, y, z, angle = stats.split(",")
    
                # Store position in the dictionary
                player_positions[run][int(tic)] = (float(x), float(y), float(z), float(angle))

In [9]:
class DoomMotionDataset(Dataset):
    def __init__(self, coco, run, transform=None):
        self.coco = coco
        self.run = run
        self.img_ids = self.coco.getImgIds()
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        # Load the RGB image
        rgb_filename = self.coco.loadImgs(self.img_ids[idx])[0]['file_name']
        tic = int(rgb_filename.replace(".png", "").split("/")[-1])
        next_tic = tic+1
        previous_tic = tic-1

        player_position = player_positions[self.run][tic]
        
        if next_tic not in player_positions[self.run]:
            next_player_position = player_position
        else:
            next_player_position = player_positions[self.run][next_tic]
        dx = next_player_position[0] - player_position[0]
        dy = next_player_position[1] - player_position[1]
        dz = next_player_position[2] - player_position[2]
        dangle = np.pi - abs(abs(next_player_position[3] - player_position[3]) - np.pi)
        
        dx_relative = dx * np.cos(2 * np.pi - player_position[3]) + dy * np.cos(player_position[3] - 1/2 * np.pi)
        dy_relative = dx * np.sin(2 * np.pi - player_position[3]) + dy * np.sin(player_position[3] - 1/2 * np.pi)
        next_motion_vector = (dx_relative, dy_relative, dz, dangle)

        if previous_tic not in player_positions[self.run]:
            prev_player_position = player_position
        else :
            prev_player_position = player_positions[self.run][previous_tic]
        dx = player_position[0] - prev_player_position[0]
        dy = player_position[1] - prev_player_position[1]
        dz = player_position[2] - prev_player_position[2]
        dangle = np.pi - abs(abs(player_position[3] - prev_player_position[3]) - np.pi)
        
        dx_relative = dx * np.cos(2 * np.pi - prev_player_position[3]) + dy * np.cos(prev_player_position[3] - 1/2 * np.pi)
        dy_relative = dx * np.sin(2 * np.pi - prev_player_position[3]) + dy * np.sin(prev_player_position[3] - 1/2 * np.pi)
        prev_motion_vector = (dx_relative, dy_relative, dz, dangle)


        # if dx > 1000:
        #     print(f"idx: {idx}")
        #     print(f"rgb_filename: {rgb_filename}")
        #     print(f"tic: {tic}")
        #     print(f"next_tic: {next_tic}")
        #     print(f"previous_tic: {previous_tic}")
        #     print(f"Sus {idx}")
        #     print(f"prev_player_position: {prev_player_position}")
        #     print(f"player_position: {player_position}")
        #     print(f"next_player_position: {next_player_position}")
        #     print(f"prev_motion_vector: {prev_motion_vector}")
        #     print(f"next_motion_vector: {next_motion_vector}")
            
        prev_motion_vector = torch.tensor(prev_motion_vector, dtype=torch.float32)
        next_motion_vector = torch.tensor(next_motion_vector, dtype=torch.float32)
        
        return prev_motion_vector, next_motion_vector


In [10]:
class NeuralNetwork(nn.Module):
  def __init__(self, activation_function=functions.relu, device=torch.device("cpu")):
    super(NeuralNetwork, self).__init__()

    self.activation_function = activation_function
    self.fc1 = nn.Linear(4, 64).to(device)
    self.bn1 = nn.BatchNorm1d(num_features=64).to(device)

    self.fc2 = nn.Linear(64, 128).to(device)
    self.bn2 = nn.BatchNorm1d(num_features=128).to(device)

    self.fc3 = nn.Linear(128, 64).to(device)
    self.bn3 = nn.BatchNorm1d(num_features=64).to(device)

    self.out = nn.Linear(64, 4).to(device)

  def forward(self, x):
    x = self.activation_function(self.bn1(self.fc1(x)))
    x = self.activation_function(self.bn2(self.fc2(x)))
    x = self.activation_function(self.bn3(self.fc3(x)))
    x = self.out(x)
    return x

In [11]:
batch_size = 256
learning_rate = 1e-3
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(device=device).to(device)

train_dataset = DoomMotionDataset(coco_train, TRAIN_RUN)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

val_dataset = DoomMotionDataset(coco_val, VAL_RUN)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
    
        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "batch_index": batch_idx + 1,
            "batch_size": inputs.size(0)
        })

    # Average loss per epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    
    
    progress_bar = tqdm(val_loader, desc="Validation", unit="batch")
    
    with torch.no_grad():  # Disable gradient calculations for evaluation
        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            
            progress_bar.set_postfix({
                "batch_loss": loss.item(),
                "batch_index": batch_idx + 1,
                "batch_size": inputs.size(0)
            })
    
    # Average loss over all batches
    val_loss = running_loss / len(val_loader)
    print(f"Val Loss: {val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "movement_nn.pth")

cpu


Epoch 1/10: 100%|████████████| 991/991 [00:18<00:00, 52.81batch/s, batch_loss=27.1, batch_index=991, batch_size=220]


Epoch 1, Loss: 482.6703


Validation: 100%|████████████| 743/743 [00:10<00:00, 72.83batch/s, batch_loss=4.52, batch_index=743, batch_size=238]


Val Loss: 638.9198


...

Val Loss: 630.2997


Epoch 10/10: 100%|███████████| 991/991 [00:18<00:00, 53.25batch/s, batch_loss=24.1, batch_index=991, batch_size=220]


Epoch 10, Loss: 475.3534


Validation: 100%|████████████| 743/743 [00:09<00:00, 77.40batch/s, batch_loss=4.63, batch_index=743, batch_size=238]


Val Loss: 627.9109


In [13]:
device = (torch.device("cuda" if torch.cuda.is_available() else "cpu"))
criterion = torch.nn.MSELoss()
model = NeuralNetwork(device="cuda").to(device)
model.load_state_dict(torch.load("movement_nn.pth", weights_only=True))

test_dataset = DoomMotionDataset(coco_test, TEST_RUN)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

model.eval()  # Set the model to evaluation mode
running_loss = 0.0


progress_bar = tqdm(test_loader, desc="Testing", unit="batch")

with torch.no_grad():  # Disable gradient calculations for evaluation
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        running_loss += loss.item()
        
        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "batch_index": batch_idx + 1,
            "batch_size": inputs.size(0)
        })

# Average loss over all batches
test_loss = running_loss / len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

AssertionError: Torch not compiled with CUDA enabled